##### Copyright 2024 Google LLC.

In [ ]:
# @title Licensed under the Apache License, Version 2.0 (the "License");
# you may not use this file except in compliance with the License.
# You may obtain a copy of the License at
#
# https://www.apache.org/licenses/LICENSE-2.0
#
# Unless required by applicable law or agreed to in writing, software
# distributed under the License is distributed on an "AS IS" BASIS,
# WITHOUT WARRANTIES OR CONDITIONS OF ANY KIND, either express or implied.
# See the License for the specific language governing permissions and
# limitations under the License.

# Gemini API: LLM Observability with MLflow

<table class="tfo-notebook-buttons" align="left">
  <td>
    <a target="_blank" href="https://colab.research.google.com/github/google-gemini/cookbook/blob/main/examples/mlflow/MLflow_Observability.ipynb"><img src="../../images/colab_logo_32px.png" />Run in Google Colab</a>
  </td>
</table>


## Overview

[MLflow](https://mlflow.org/) is an open-source platform to assist machine learning practitioners and teams in handling the complexities of the machine learning process.

It provides [MLflow Tracing](https://mlflow.org/docs/latest/tracing/) that enhances LLM observability in your Generative AI applications by capturing detailed information about the execution of your application’s services. Tracing provides a way to record the inputs, outputs, and metadata associated with each intermediate step of a request, enabling you to easily pinpoint the source of bugs and unexpected behaviors.

MLflow provides a built-in integration with Google Gen AI SDK that enables you to instrument your Gemini calls easily. This cookbook describes the basic usage of the MLflow tracing integration with the `google-genai` package.

<!-- Community Contributor Badge -->
<table>
  <tr>
    <!-- Author Avatar Cell -->
    <td bgcolor="#d7e6ff">
      <a href="https://github.com/TomeHirata" target="_blank" title="View Tomu Hirata's profile on GitHub">
        <img src="https://github.com/TomeHirata.png?size=100"
             alt="TomeHirata's GitHub avatar"
             width="100"
             height="100">
      </a>
    </td>
    <!-- Text Content Cell -->
    <td bgcolor="#d7e6ff">
      <h2><font color='black'>This notebook was contributed by <a href="https://github.com/TomeHirata" target="_blank"><font color='#217bfe'><strong>Tomu Hirata</strong></font></a>.</font></h2>
      <!-- Footer -->
      <font color='black'><small><em>Have a cool Gemini example? Feel free to <a href="https://github.com/google-gemini/cookbook/blob/main/CONTRIBUTING.md" target="_blank"><font color="#078efb">share it too</font></a>!</em></small></font>
    </td>
  </tr>
</table>

## Installation
Install Google Gen AI SDK (`google-genai`) and MLflow (`mlflow`). See [troubleshooting for MLFlow installation](https://mlflow.org/docs/latest/quickstart_drilldown/#quickstart_drilldown_install) for other install options.

In [2]:
# Integration with google-genai is supported mlflow >= 2.20.3
%pip install -q google-genai "mlflow>=2.20.3"


[notice] A new release of pip is available: 24.3.1 -> 25.0.1
[notice] To update, run: pip install --upgrade pip
Note: you may need to restart the kernel to use updated packages.


## Create Gemini Client with your API key

Let's create an API client and pass your API key. If you do not have API ket yet, visit [AI Studio](https://aistudio.google.com/app/apikey) to create one.

In [3]:
import google.genai as genai
from google.colab import userdata

client = genai.Client(api_key=userdata.get('GOOGLE_API_KEY'))

## Tracking Server
There are several options to run MLflow tracking server: local tracking server, Databricks Free Trial, and production Databricks. See [our documentation](https://mlflow.org/docs/latest/getting-started/tracking-server-overview/index.html) for the comparison. In this example, you use [Databricks Free Trial](https://mlflow.org/docs/latest/getting-started/databricks-trial/) which allows easy connection from Colab notebooks and enable you to use managed MLflow for free. Follow the steps below to create an account and generate a Personal Access Token (PAT) to connect to your workspace.

* Go to the [Databricks Trial Signup Page](http://signup.databricks.com/) and create your account
* Follow the steps in [this guide](https://docs.databricks.com/aws/en/dev-tools/auth/pat) to create a PAT for your Databricks workspace user
* You need following information to connect to your workspace from your notebook
  * Databricks Host: Use "https://\<your workspace host\>.cloud.databricks.com/
  * Token: Your personal access token for your Databricks Workspace. 

In [4]:
WORKSPACE_EMAIL = userdata.get("WORKSPACE_EMAIL") # your email
WORKSPACE_PAT = userdata.get("WORKSPACE_PAT") # your databricks host https://<your workspace host>.cloud.databricks.com/
WORKSPACE_URL = userdata.get("WORKSPACE_URL") # your PAT

Then, set the tracking server uri and experiment name.

In [ ]:
import os
import mlflow
mlflow.set_tracking_uri('databricks')
mlflow.set_registry_uri('databricks-uc')
os.environ['DATABRICKS_HOST'] = WORKSPACE_URL
os.environ['DATABRICKS_TOKEN'] = WORKSPACE_PAT
mlflow.login()

In [ ]:
# Create Experiment
mlflow.create_experiment(
    f"/Users/{WORKSPACE_EMAIL}/gemini",
    artifact_location="dbfs:/Volumes/workspace/default/gemini",
)
mlflow.set_experiment(f"/Users/{WORKSPACE_EMAIL}/gemini")

## Enable AutoLogging
MLflow Tracing provides automatic tracing capability for Google Gemini. By enabling auto tracing for Gemini by calling the `mlflow.gemini.autolog()` function, MLflow will capture nested traces and log them to the active MLflow Experiment upon invocation of Gemini Python SDK.

MLflow trace automatically captures the following information about Gemini calls:
- Prompts and completion responses
- Latencies
- Model name
- Additional metadata such as temperature, max_tokens, if specified.
- Function calling if returned in the response
- Any exception if raised

In [7]:
import mlflow

# Turn on auto tracing for Gemini
mlflow.gemini.autolog()

## Call Simple Content Generation
Let's run `client.models.generate_content` to try a simple text generation use case with MLflow tracing. For Jupyter Notebook users, MLflow provides a convenient way to see the generated traces on your notebook. See [this blog](https://mlflow.org/blog/mlflow-tracing-in-jupyter) for more information. For users who use other platforms, visit "http://localhost:5000" to see MLflow UI.

In [ ]:
MODEL_ID = "gemini-2.5-flash-preview-05-20" # @param ["gemini-2.0-flash-lite","gemini-2.0-flash","gemini-2.5-flash-preview-05-20","gemini-2.5-pro-exp-03-25"] {"allow-input":true, isTemplate: true}

In [17]:
result = client.models.generate_content(
    model=MODEL_ID, contents="The opposite of hot is"
)
result.text

'cold\n'

<img src="https://i.imgur.com/xWGZXtZ.png" />

## Multi-Turn Chat Interactions
MLflow tracing captures the structure of your interactions with the Gemini API. Run the following cell to try multi-turn chat and see how MLflow captures the interaction.

In [16]:
with mlflow.start_span(name="multi-turn"):
    chat = client.chats.create(model=MODEL_ID)
    response = chat.send_message("In one sentence, explain how a computer works to a young child.")
    print(response.text)
    response = chat.send_message("Okay, how about a more detailed explanation to a high schooler?")
    print(response.text)

A computer follows instructions, like a recipe, to do things with numbers and pictures.

A computer works by executing a sequence of instructions, called a program, written in a language it understands. These instructions manipulate binary data (0s and 1s) representing information, performing calculations, storing and retrieving data from memory and storage devices, and interacting with input and output devices like the keyboard, screen, and network to complete tasks.



<img src="https://i.imgur.com/ftJ3CEp.png">

## Function Call
If your application uses [function calling](https://ai.google.dev/gemini-api/docs/function-calling) of the Gemini API, the function definition, function call and function response are captured by MLflow automatically.

In [18]:
def add(a: float, b: float):
    """returns a + b."""
    return a + b


def subtract(a: float, b: float):
    """returns a - b."""
    return a - b


def multiply(a: float, b: float):
    """returns a * b."""
    return a * b


def divide(a: float, b: float):
    """returns a / b."""
    return a / b


operation_tools = [add, subtract, multiply, divide]

In [19]:
chat = client.chats.create(
    model = MODEL_ID,
    config = {
        "tools": operation_tools,
    }
)

In [24]:
response = chat.send_message(
    "I have 57 cats, each owns 44 mittens, how many mittens is that in total?"
)
response.text

'2508\n'

<img src="https://i.imgur.com/mJeryzg.png"/>

## Conclusion

That's all for this cookbook. MLflow tracing provides many features that are not included in this notebook and actively releases new features. Visit [MLflow Gemini tracing integration](https://mlflow.org/docs/latest/tracing/integrations/gemini) for more configurations and [MLflow Tracing Overview](https://mlflow.org/docs/latest/tracing/) for general offerings.